# Assignment: Coursework I
## Group 8
1. Maxwell Sarpong (10935738)
2. Clemence Ayekple (10936515)
3. Sarah Quaye (10937270)
4. Allan Philip Barku (10936490)
5. Bernard Zottorvi (10938772)
6. Dodzi Kwame Kumah(10936281)
7. Kassah Machel Sedinam(10932715)


## 1 A. File Retrieval Exercise 5 Marks
Write a Java code or any relevant programming code using Remote Method Invocation (RMI) system,  which retrieves a file specified by a Client from the Server. You are expected to follow the following general steps when using the Java RMI or Python Remote Object (Pyro4).
> a) Writing the required remote interface 1 Mark

> b) Implementing the interface 1 Mark

> c) Creating stub and skeleton (Compiling remote interface and its implementation) 1 Mark

> d) Starting the registry mechanism

> e) Writing the main Client and Server programs 1 Mark

> f) Execution of the Client and Server to show the required output 1 Mark

The contents of the retrieved file may be saved in another file or may be printed on the screen.


In [ ]:
%pip install Pyro4
%pip install rsa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 3.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# importing Pyro4 library
import Pyro4
import Pyro4.util

In [ ]:
# remote interface
class StudentGradeInterface:
    def list_all_grades(self) -> dict:
        pass

    def retrieve_single_grade(self, name, student_id) -> str:
        pass


In [ ]:
# server side with the implemention of the interface

@Pyro4.expose
@Pyro4.behavior(instance_mode="single")
class StudentGrade(StudentGradeInterface):
    def __init__(self):
        self.contents = [{"student_id": 109864, "name": "John Doe", "grades": {
            "Maths": "A", "Science": "C"}}, {"student_id": 109764, "name": "Peter Adjei", "grades": {
                "Maths": "A", "Science": "A"}}, {"student_id": 109863, "name": "Larry Bell", "grades": {
                    "Maths": "C", "Science": "B"}}]

    def list_all_grades(self) -> dict:
        return self.contents

    def retrieve_single_grade(self, name, item) -> str:
        next(
            (grade for grade in self.contents if grade["student_id"] == item), False)
        print("{0} retrieved grade for {1}.".format(name, item))


def main():
    Pyro4.Daemon.serveSimple(
        {
            StudentGrade: "example.studentgrade"
        }, ns=False)


if __name__ == "__main__":
    main()


Object <class '__main__.StudentGrade'>:
    uri = PYRO:example.studentgrade@localhost:37389
Pyro daemon running.


In [ ]:
# client side

class Student(object):
    def __init__(self, name):
        self.name = name

    def grades(self, studentgrade):
        print("This is {0}.".format(self.name))
        self.retrieve(studentgrade)
        print("Thank you, come again!")


    def retrieve(self, studentgrade):
        print("The student grades:", studentgrade.list_all_grades())
        item = input("Enter student id: ").strip()
        if item:
            studentgrade.retrieve_single_grade(self.name, item)


In [ ]:
uri = input("Enter the uri of the grade: ").strip()
grade = Pyro4.Proxy(uri)
janet = Student("Janet")
janet.grades(grade)

Enter the uri of the grade: PYRO:example.studentgrade@localhost:60219

This is Janet.

The student grades: [{'student_id': 109864, 'name': 'John Doe', 'grades': {'Maths': 'A', 'Science': 'C'}}, {'student_id': 109764, 'name': 'Peter Adjei', 'grades': {'Maths': 'A', 'Science': 'A'}}, {'student_id': 109863, 'name': 'Larry Bell', 'grades': {'Maths': 'C', 'Science': 'B'}}]

Enter student id: 109863

Thank you, come again!

## 1 B. Secure File Retrieval Exercise 15 Marks

Modify the code of the exercise 1A and include additional code wherever appropriate so that, the interaction between the Client and Server is secure. Along with RMISecurityManager, use RSA encryption algorithm to generate the digital signature and for secure Client authentication. The following steps are expected and evaluated as shown.
> a) Creating a key store and generating a pair of public and private keys 3 Marks

> b) Exporting a digital certificate for the above key into a file 2 Marks

> c) Creating a trust store and import the certificate as a trusted certificate 3 Marks

>d) Execution of the final program and presentation 5 Marks

The contents of the retrieved file may be saved in another file or may be printed on the screen. Upon program execution you are required to explain briefly all the key in-built classes and methods used to
implement. 2 Marks

Q. What changes are required to implement the above program in a physical network rather than a single machine?

In [ ]:
#importing the rsa library
import rsa

In [ ]:
# generating public and private keys using the rsa library
def generate_keys():
    (publicKey, privateKey) = rsa.newkeys(1024)
    with open('keys/publicKey.pem', 'wb') as p:
        p.write(publicKey.save_pkcs1('PEM'))
    with open('keys/privateKey.pem', 'wb') as p:
        p.write(privateKey.save_pkcs1('PEM'))

In [ ]:
#load the encryption key
def load_keys():
    with open('keys/publicKey.pem', 'rb') as p:
        publicKey = rsa.PublicKey.load_pkcs1(p.read())
    with open('keys/privateKey.pem', 'rb') as p:
        privateKey = rsa.PrivateKey.load_pkcs1(p.read())
    return privateKey, publicKey

In [ ]:
# encrypting the message
def encrypt(message, key):
    return rsa.encrypt(message.encode('ascii'), key)

# decrypting the message
def decrypt(ciphertext, key):
    try:
        return rsa.decrypt(ciphertext, key).decode('ascii')
    except:
        return False

# signing the message
def sign(message, key):
    return rsa.sign(message.encode('ascii'), key, 'SHA-1')

# verying the message
def verify(message, signature, key):
    try:
        return rsa.verify(message.encode('ascii'), signature, key,) == 'SHA-1'
    except:
        return False

In [ ]:
generate_keys()
publicKey, privateKey =load_keys()

In [ ]:
#server side modification
@Pyro4.expose
@Pyro4.behavior(instance_mode="single")
class StudentGrade(StudentGradeInterface):
    def __init__(self):
        self.contents = [{"student_id": 109864, "name": "John Doe", "grades": {
            "Maths": "A", "Science": "C"}}, {"student_id": 109764, "name": "Peter Adjei", "grades": {
                "Maths": "A", "Science": "A"}}, {"student_id": 109863, "name": "Larry Bell", "grades": {
                    "Maths": "C", "Science": "B"}}]

    def list_all_grades(self) -> dict:
        return self.contents

    def retrieve_single_grade(self, name, item) -> str:
        grade = next(
            (grade for grade in self.contents if grade["student_id"] == item), False
            )
        #encrypting and creating a digital signature
        ciphertext = encrypt(grade, publicKey)
        signature = sign(grade, privateKey)
        print("{0} retrieved grade for {1}.".format(name, item))


def main():
    Pyro4.Daemon.serveSimple(
        {
            StudentGrade: "example.studentgrade"
        }, ns=False)


if __name__ == "__main__":
    main()

In [ ]:
#client side modification
class Student(object):
    def __init__(self, name):
        self.name = name

    def grades(self, studentgrade):
        print("This is {0}.".format(self.name))
        self.retrieve(studentgrade)
        text = decrypt(studentgrade, privateKey)
        print("Thank you, come again!")


    def retrieve(self, studentgrade):
        print("The student grades:", studentgrade.list_all_grades())
        item = input("Enter student id: ").strip()
        if item:
            studentgrade.retrieve_single_grade(self.name, item)

**Q. What changes are required to implement the above program in a physical network rather than a single machine?**

Pyro runs on localhost by default. To access it on a different machine you will need to tell Pyro to do that explicitly.

This can be done in the console as shown below:

    $ python -m Pyro4.naming -n your_hostname # i.e. your_hostname = “192.168.10.9”

or simply:

    $ pyro4-ns -n your_hostname

This can also be implemented in the code by specifying it in the models below:

Pyro4.config.HOST = "your_hostname_here"
Pyro4.Daemon.serveSimple(...)

Optionally, you can set PYRO_HOST environment variable before starting the server side of the application. Another choice is to pass the required host (and perhaps even port) arguments to serveSimple:

Pyro4.Daemon.serveSimple(
        {
            Warehouse: "example.warehouse"
        },
        host = 'your_hostname_here',
        ns = True)

